In [9]:
import gurobipy as gp
from gurobipy import GRB

# The Model

In [22]:
theater_seat_organizer = gp.Model()

# The Parameters

In [80]:
number_of_customers_in_group = [3, 5, 6] 

customer_vaccination_status = [
    [0,0,0], 
    [1,1,1,1,1],
    [0,1,0,0,1,1]
]

number_of_seats = 4

non_vax_cost = 5

# Parameter Validation Logic

In [88]:
def isBinary(num):
    if(num == 1 or num == 0):
        return True
    else:
        return False


# Vaccine Records Should Match Customer Records
number_of_vax = count = sum( [ len(customers) for customers in customer_vaccination_status])

if(sum(number_of_customers_in_group) != number_of_vax):
    raise ValueError('Vaccine Records Do Not Match Customer Records')
    
# Vaccine Records Should be Formatted like Customer Records
for index, customer_ofGroup_vaxList in enumerate(customer_vaccination_status):
    if any(not isBinary(person) for person in customer_ofGroup_vaxList):
        raise ValueError('Invalid Arg: Person is either vaxed = 1 or unvaxxed = 0')
        
    if(len(customer_ofGroup_vaxList) != number_of_customers_in_group[index]):
        raise ValueError('Vaccine Records Are Not Formatted Correctly')



# Decision Variables

In [65]:
person_inGroup_atSeat = {}

for group_num in range(len(number_of_customers_in_group)):
    for customer_index in range(0, number_of_customers_in_group[group_num]):
        for seat in range(number_of_seats):
            
            person_inGroup_atSeat[customer_index, group_num, seat] = theater_seat_organizer.addVar(vtype = GRB.BINARY, name = "person" + str(customer_index) + "_inGroup" + str(group_num) + "_atSeat" + str(seat))

groupAccepted = {}
for group_num in range(len(number_of_customers_in_group)):
    groupAccepted[group_num] = theater_seat_organizer.addVar(vtype = GRB.BINARY, name = "group" + str(group_num) + "_admitted")


            

# The Model

In [66]:
objectiveFunction = gp.LinExpr()

for group_num in range(len(number_of_customers_in_group)):
    for customer_index in range(0, number_of_customers_in_group[group_num]):
        for seat in range(number_of_seats):
            objectiveFunction += person_inGroup_atSeat[customer_index, group_num, seat]   
    
for group_num in range(len(number_of_customers_in_group)):
    objectiveFunction -= non_vax_cost * (1-groupAccepted[group_num])

theater_seat_organizer.setObjective(objectiveFunction,  GRB.MAXIMIZE)

21